# Qwen1.5-0.5B-Chat Test Notebook

基于香橙派AIpro + MindSpore 实现的通义千问Qwen聊天机器人测试笔记本

**环境要求:**
- CANN 8.1.RC1
- MindSpore 2.6.0  
- mindnlp 0.4.1

**参考:** [昇腾社区 - 基于香橙派AIpro+MindSpore实现Qwen聊天机器人](https://www.hiascend.com/developer/techArticles/20250424-3)

## 1. 环境检查

首先检查系统环境是否正确配置。

In [ ]:
# Check MindSpore version
import mindspore
print(f"MindSpore version: {mindspore.__version__}")

# Check mindnlp version
import mindnlp
print(f"mindnlp version: {mindnlp.__version__}")

# Check NPU availability
from mindspore import context
print(f"MindSpore device target: {context.get_context('device_target')}")

In [ ]:
# Check available memory
import psutil
mem = psutil.virtual_memory()
print(f"Total Memory: {mem.total / (1024**3):.2f} GB")
print(f"Available Memory: {mem.available / (1024**3):.2f} GB")
print(f"Memory Used: {mem.percent}%")

## 2. 模型配置

设置模型名称和数据类型。

In [ ]:
# Model configuration
MODEL_NAME = "Qwen/Qwen1.5-0.5B-Chat"
MS_DTYPE = mindspore.float16  # Use FP16 for memory efficiency

# Generation parameters
MAX_NEW_TOKENS = 1024
TEMPERATURE = 0.1
TOP_P = 0.9
DO_SAMPLE = True

print(f"Model: {MODEL_NAME}")
print(f"Data type: {MS_DTYPE}")
print(f"Max new tokens: {MAX_NEW_TOKENS}")

## 3. 加载模型和分词器

**注意:** 首次运行需要从 Hugging Face 下载模型 (约 1GB)，可能需要几分钟时间。

如果下载速度较慢，可以设置镜像源:
```bash
export HF_ENDPOINT=https://hf-mirror.com
```

In [ ]:
from mindnlp.transformers import AutoModelForCausalLM, AutoTokenizer

print(f"Loading tokenizer for {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, ms_dtype=MS_DTYPE)
print("Tokenizer loaded successfully!")

print(f"\nLoading model {MODEL_NAME}...")
print("(This may take 1-2 minutes on first run)")
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, ms_dtype=MS_DTYPE)
print("Model loaded successfully!")

In [ ]:
# Check model architecture
print(f"Model type: {type(model).__name__}")
print(f"Model config:\n{model.config}")

## 4. 定义聊天函数

In [ ]:
from mindnlp.transformers import TextIteratorStreamer
from threading import Thread

# System prompt - defines the bot's personality
SYSTEM_PROMPT = "You are a helpful and friendly chatbot"

def build_input_from_chat_history(chat_history, msg: str):
    """Build message list from chat history and new message."""
    messages = [{'role': 'system', 'content': SYSTEM_PROMPT}]
    for user_msg, ai_msg in chat_history:
        messages.append({'role': 'user', 'content': user_msg})
        messages.append({'role': 'assistant', 'content': ai_msg})
    messages.append({'role': 'user', 'content': msg})
    return messages

def generate_response(message, chat_history=None, stream=False):
    """Generate a response from the model.
    
    Args:
        message: User's input message
        chat_history: List of (user_msg, ai_msg) tuples
        stream: If True, yield partial responses as they're generated
    
    Returns:
        If stream=True: Generator yielding partial responses
        If stream=False: Complete response string
    """
    if chat_history is None:
        chat_history = []
    
    # Format messages for the model
    messages = build_input_from_chat_history(chat_history, message)
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="ms",
        tokenize=True
    )
    
    if stream:
        # Streaming mode - yield partial responses
        streamer = TextIteratorStreamer(
            tokenizer, 
            timeout=300, 
            skip_prompt=True, 
            skip_special_tokens=True
        )
        generate_kwargs = dict(
            input_ids=input_ids,
            streamer=streamer,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=DO_SAMPLE,
            top_p=TOP_P,
            temperature=TEMPERATURE,
            num_beams=1,
        )
        t = Thread(target=model.generate, kwargs=generate_kwargs)
        t.start()
        
        partial_message = ""
        for new_token in streamer:
            partial_message += new_token
            if '</s>' in partial_message:
                break
            yield partial_message
    else:
        # Non-streaming mode - return complete response
        outputs = model.generate(
            input_ids,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=DO_SAMPLE,
            top_p=TOP_P,
            temperature=TEMPERATURE,
        )
        response = tokenizer.decode(outputs[0][input_ids.shape[1]:], skip_special_tokens=True)
        return response

print("Chat functions defined successfully!")

## 5. 简单测试 - 无上下文

测试模型对单个问题的响应。

In [ ]:
# Simple test - single question, no context
test_question = "你好，请介绍一下你自己。"

print(f"Question: {test_question}")
print("\nGenerating response...")

response = generate_response(test_question)

print(f"\nResponse: {response}")

## 6. 流式输出测试

测试流式输出 - 逐token显示生成过程。

In [ ]:
# Streaming test
test_question = "用Python写一个计算斐波那契数列的函数"

print(f"Question: {test_question}")
print("\nResponse (streaming):\n")

full_response = ""
for partial in generate_response(test_question, stream=True):
    # Clear the line and print the new partial response
    print(f"\r{partial}", end="", flush=True)
    full_response = partial

print(f"\n\nFinal response:\n{full_response}")

## 7. 多轮对话测试

测试多轮对话能力。

In [ ]:
# Multi-turn conversation test
chat_history = []

def chat_turn(user_message, history):
    """Run one turn of the conversation."""
    print(f"\n{'='*60}")
    print(f"User: {user_message}")
    
    response = generate_response(user_message, history)
    
    print(f"\nAssistant: {response}")
    
    # Update history
    history.append((user_message, response))
    return history

# Conversation turns
chat_history = chat_turn("你好，我叫小明。", chat_history)
chat_history = chat_turn("我叫什么名字？", chat_history)
chat_history = chat_turn("你能帮我做什么？", chat_history)

## 8. 代码生成测试

测试模型的代码生成能力。

In [ ]:
# Code generation test
code_questions = [
    "用Python写一个冒泡排序函数",
    "解释一下什么是递归，并给出一个例子",
    "如何用Python读取CSV文件？"
]

for question in code_questions:
    print(f"\n{'='*60}")
    print(f"Question: {question}")
    print("\nResponse:")
    
    response = generate_response(question)
    print(response)

## 9. 不同温度参数测试

测试不同温度参数对生成结果的影响。

- **Temperature = 0.1**: 更确定性的输出
- **Temperature = 0.7**: 平衡的创造性和一致性
- **Temperature = 1.0**: 更随机、更有创造性的输出

In [ ]:
# Temperature parameter test
test_prompt = "写一个关于AI的小故事"
temperatures = [0.1, 0.7, 1.0]

for temp in temperatures:
    # Update global temperature
    globals()['TEMPERATURE'] = temp
    
    print(f"\n{'='*60}")
    print(f"Temperature = {temp}")
    print(f"Prompt: {test_prompt}")
    print("\nResponse:")
    
    response = generate_response(test_prompt)
    print(response)

# Reset temperature to default
TEMPERATURE = 0.1

## 10. 交互式聊天测试

在下面的单元格中输入你自己的问题进行测试。

In [ ]:
# Interactive chat - modify the question and run this cell
user_question = """  # <- 在这里输入你的问题
"""

if user_question.strip():
    print(f"Question: {user_question}")
    print("\nResponse:")
    response = generate_response(user_question)
    print(response)
else:
    print("Please enter a question in the user_question variable.")

## 11. 性能测试

测量模型推理的响应时间和内存使用。

In [ ]:
import time
import psutil

# Performance test
test_questions = [
    "你好",
    "介绍一下Python编程语言",
    "什么是机器学习？"
]

process = psutil.Process()

print("Performance Test Results")
print("="*60)
print(f"{'Question':<30} {'Time (s)':<12} {'Memory (MB)':<12}")
print("-"*60)

for question in test_questions:
    # Get initial memory
    mem_before = process.memory_info().rss / (1024 * 1024)
    
    # Measure time
    start_time = time.time()
    response = generate_response(question)
    elapsed = time.time() - start_time
    
    # Get final memory
    mem_after = process.memory_info().rss / (1024 * 1024)
    mem_used = mem_after - mem_before
    
    # Print results
    print(f"{question[:28]:<30} {elapsed:<12.2f} {mem_used:<12.2f}")
    print(f"Response: {response[:50]}...\n")

## 12. Gradio界面启动（可选）

如果你想启动完整的Gradio聊天界面，运行下面的单元格。

启动后在浏览器中打开: http://127.0.0.1:7860/

In [ ]:
# Optional: Launch Gradio interface
# Uncomment the lines below to launch the web interface

# import gradio as gr

# def gradio_predict(message, history):
#     for partial in generate_response(message, history, stream=True):
#         yield partial

# demo = gr.ChatInterface(
#     gradio_predict,
#     title="Qwen1.5-0.5B-Chat on Orange Pi AI Pro",
#     description="基于MindSpore + NPU的Qwen聊天机器人",
#     examples=['你是谁？', '介绍一下Redhat公司', '用Python写一个快速排序']
# )

# demo.launch()

print("Gradio launch code is commented out by default.")
print("Uncomment the code above to launch the web interface.")

## 测试总结

如果所有测试都通过，说明你的环境已经正确配置，可以正常运行Qwen聊天机器人。

### 预期性能指标:

| 指标 | 预期值 |
|------|--------|
| 模型加载时间 | 1-2 分钟 |
| 首次响应时间 | 10-30 秒 |
| 后续响应时间 | 5-15 秒 |
| 内存占用 | 2-3 GB |

### 常见问题:

**Q: 首次运行很慢？**
A: 需要从Hugging Face下载模型，约1GB数据。下载后会缓存。

**Q: 内存不足？**
A: 关闭其他程序，或使用更小的模型。

**Q: NPU错误？**
A: 确保CANN环境正确设置: `source /usr/local/Ascend/ascend-toolkit/set_env.sh`